In [ ]:
vgg_local = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(*SIZE, 3)))
vgg_local.trainable = False

flatten_bbox_local = Flatten()(vgg_local.output)
dense_1_bbox_local = Dense(128, activation = 'relu')(flatten_bbox_local)
dense_2_bbox_local = Dense(32, activation = 'relu')(dense_1_bbox_local)
output_bbox_local = Dense(4, activation = lambda x: 200*tf.keras.activations.sigmoid(x))(dense_2_bbox_local)
model_bbox_base_local = Model(inputs = vgg_local.inputs, outputs = output_bbox_local)

model_bbox_base_local.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001), loss = 'mse')
model_bbox_base_local.load_weights('modelo_bbox_mse_weights')

@tf.function
def parse_ds_reg_cut_function(example_input: Optional[tf.Tensor],) -> Optional[Tuple[tf.Tensor]]:
    image = tf.io.read_file(example_input[0])
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, SIZE)
    image = tf.cast(image, tf.float32) / 255.

    bbox_predict = model_bbox_base_local(tf.expand_dims(image, axis=0))
    image = tf.image.crop_and_resize(tf.expand_dims(image, axis=0), 
                                     tf.math.divide(bbox_predict, tf.constant(200.)), 
                                     tf.random.uniform(shape=(1,), minval=0, maxval=1, dtype=tf.int32), 
                                     SIZE)

    val = tf.strings.to_number(example_input[1])
    exc = tf.strings.to_number(example_input[2])
    dom = tf.strings.to_number(example_input[3])

    return tf.squeeze(image), [val, exc, dom]

In [ ]:
train_ds_reg_cut = train_list_ds_reg.map(parse_ds_reg_cut_function)
val_ds_reg_cut = val_list_ds_reg.map(parse_ds_reg_cut_function)
test_ds_reg_cut = test_list_ds_reg.map(parse_ds_reg_cut_function)

In [ ]:
train_ds_reg_cut  = performance(train_ds_reg_cut)
val_ds_reg_cut = performance(val_ds_reg_cut, False)
test_ds_reg_cut = performance(test_ds_reg_cut, False)

In [ ]:
for X_batch, y_batch in val_ds_reg_cut.take(1):
    break
np.random.seed(0)
random_sample = np.random.choice(len(X_batch), 9)
X_img = X_batch.numpy()[random_sample]

In [ ]:
visualize_subplot([img for img in X_img], ["" for _ in X_img], (3, 3), (10, 10),)